## 대성에너지 주가 동향 Daily Report

In [51]:
import requests 
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta

ticker = '117580'   # 대성에너지

### 1. 주식 정보 가져오기

In [52]:
# fnGuide 주식 정보
url = f"http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{ticker}"
resp = requests.get(url)

if resp.status_code == 200:
    soup = BeautifulSoup(resp.text, "lxml")
    
    # 시세현황 기준일자
    selector_ref_date = '#div1 > div.um_topbar.botbd_n.bmshadow > div > div.topbar_lft > span'
    # 종가/ 전일대비
    selector_close_price = '#svdMainGrid1 > table > tbody > tr.rwf > td:nth-child(2)'
    # 52주 최고가/최저가
    selector_one_year = '#svdMainGrid1 > table > tbody > tr:nth-child(2) > td:nth-child(2)'
    # 시가총액(보통주, 억원)
    selector_market_cap = '#svdMainGrid1 > table > tbody > tr:nth-child(5) > td:nth-child(2)'
    # 거래량
    selector_volume = '#svdMainGrid1 > table > tbody > tr.rwf > td.cle.r'
    # 거래대금(억원)
    selector_tr_amt = '#svdMainGrid1 > table > tbody > tr:nth-child(2) > td.cle.r'
    # 외국인 보유비중(%)
    selector_foreigner = '#svdMainGrid1 > table > tbody > tr:nth-child(3) > td.cle.r'

    v_ref_date = soup.select_one(selector_ref_date).get_text()
    v_close_price = soup.select_one(selector_close_price).get_text()
    v_one_year = soup.select_one(selector_one_year).get_text()
    v_market_cap = soup.select_one(selector_market_cap).get_text()
    v_volume = soup.select_one(selector_volume).get_text()
    v_tr_amt = soup.select_one(selector_tr_amt).get_text()
    v_foreigner = soup.select_one(selector_foreigner).get_text()    

In [53]:
v_ref_date, v_close_price, v_one_year, v_market_cap, v_volume, v_tr_amt, v_foreigner

('[2023/07/18]',
 '7,740/ -60',
 '15,500/ 7,740',
 '2,129',
 ' 42,671',
 ' 3',
 '0.48')

### 2. 주식 가격 가져오기 (6개월)

In [54]:
year, month, day = v_ref_date.replace('[', '').replace(']', '').split('/')

# today = dt.datetime.now()                         # Today
# date_to = today.strftime("%Y%m%d")
# past_date = today + relativedelta(months=-6)      # 6개월 전

ref_date = year + month + day                           # 기준일
dt_ref_date = datetime.strptime(ref_date, "%Y%m%d")
dt_past_date = dt_ref_date + relativedelta(months=-6)    # 6개월 전
date_from = dt_past_date.strftime('%Y%m%d')
date_to = ref_date

In [55]:
date_from, date_to

('20230118', '20230718')

In [77]:
# pip install pykrx
from pykrx import stock

# 해당 종목의 지정한 기간의 주식 거래 현황
df_period_price = stock.get_market_ohlcv(date_from, date_to, ticker) 

In [78]:
df_period_price = df_period_price.reset_index()
df_period_price = df_period_price.astype({'날짜' : 'string'})

In [125]:
max_row = df_period_price.shape[0]

In [79]:
df_period_price.dtypes

날짜      string[python]
시가               int32
고가               int32
저가               int32
종가               int32
거래량              int32
거래대금             int64
등락률            float32
dtype: object

In [90]:
df_recent_price = df_period_price.tail(5).sort_values('날짜', ascending=False).reset_index(drop=True)

In [91]:
df_recent_price

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률
0,2023-07-18,7770,7850,7710,7740,42671,331198060,-0.77
1,2023-07-17,7750,7900,7730,7800,50242,391124830,0.13
2,2023-07-14,7870,7870,7740,7790,42639,331877980,-0.64
3,2023-07-13,7830,7940,7830,7840,46989,369399500,0.13
4,2023-07-12,7860,7950,7780,7830,35778,280653850,0.64


### 3. 엑셀로 저장

In [135]:
# pip install openpyxl
from openpyxl import Workbook
from openpyxl.chart import Reference, LineChart, BarChart
from openpyxl.styles import Font, Border, Side, PatternFill, Alignment
import datetime

wb = Workbook()
ws = wb.active  # 현재 활성화된 sheet 가져옴
ws.title = "DailyReport"

# 열 너비 지정
ws.column_dimensions["A"].width = 22     
ws.column_dimensions["B"].width = 8     
ws.column_dimensions["C"].width = 8    
ws.column_dimensions["D"].width = 18    
ws.column_dimensions["E"].width = 16    

# ws["A1"] = 1
ROW_TITLE = 1
ROW_NO_1 = 3
ROW_NO_2 = 9
ROW_NO_3 = 18

# 행 높이
ws.row_dimensions[ROW_TITLE].height = 40        # 제목 행 높이 지정
ws.row_dimensions[ROW_NO_1].height = 25        # 소제목 행 높이 지정
ws.row_dimensions[ROW_NO_2].height = 25        # 소제목 행 높이 지정
ws.row_dimensions[ROW_NO_3].height = 25        # 소제목 행 높이 지정

title = ws.cell(row=ROW_TITLE, column=1)
sub_1 = ws.cell(row=ROW_NO_1, column=1)
sub_2 = ws.cell(row=ROW_NO_2, column=1)
sub_3 = ws.cell(row=ROW_NO_3, column=1)
ref_1 = ws.cell(row=ROW_NO_1, column=5)     # 출처 fnGuide

# Font 스타일 적용
title.font = Font(color="0000FF", size=20, bold=True)
sub_1.font = Font(size=13, italic=True, bold=True)
sub_2.font = Font(size=13, italic=True, bold=True)
sub_3.font = Font(size=13, italic=True, bold=True)
ref_1.font = Font(color="0000FF", size=9, underline="single")

# 정렬
title.alignment = Alignment(horizontal="left", vertical="center")
sub_1.alignment = Alignment(horizontal="left", vertical="center")
sub_2.alignment = Alignment(horizontal="left", vertical="center")
sub_3.alignment = Alignment(horizontal="left", vertical="center")
ref_1.alignment = Alignment(horizontal="right", vertical="bottom")

# 테두리 적용
thin_border = Border(left=Side(style="thin"),
                     right=Side(style="thin"),
                     top=Side(style="thin"),
                     bottom=Side(style="thin"))

for col in range(1, 6):
    ws.cell(row=ROW_NO_1+2, column=col).border = thin_border
    ws.cell(row=ROW_NO_1+3, column=col).border = thin_border
    ws.cell(row=ROW_NO_1+4, column=col).border = thin_border
    ws.cell(row=ROW_NO_2+2, column=col).border = thin_border
    ws.cell(row=ROW_NO_2+3, column=col).border = thin_border
    ws.cell(row=ROW_NO_2+4, column=col).border = thin_border
    ws.cell(row=ROW_NO_2+5, column=col).border = thin_border
    ws.cell(row=ROW_NO_2+6, column=col).border = thin_border
    ws.cell(row=ROW_NO_2+7, column=col).border = thin_border
    
# 셀 병합
ws.merge_cells("B5:C5")
ws.merge_cells("B6:C6")
ws.merge_cells("B7:C7")

ws.cell(row=ROW_TITLE, column=1, value="대성에너지 일일 주가 동향 "+v_ref_date)

ws.cell(row=ROW_NO_1, column=1, value="1. 일일 현황")
ws.cell(row=ROW_NO_1, column=5, value="(출처: fnGuide)")

ws.cell(row=ROW_NO_1+2, column=1, value="종가/ 전일대비")
ws.cell(row=ROW_NO_1+2, column=2).value = v_close_price
ws.cell(row=ROW_NO_1+2, column=2).alignment = Alignment(horizontal="right", vertical="center")
ws.cell(row=ROW_NO_1+3, column=1, value="52주 최고/최저")
ws.cell(row=ROW_NO_1+3, column=2).value = v_one_year
ws.cell(row=ROW_NO_1+3, column=2).alignment = Alignment(horizontal="right", vertical="center")
ws.cell(row=ROW_NO_1+4, column=1, value="시가총액(보통주, 억원)")
ws.cell(row=ROW_NO_1+4, column=2).value = v_market_cap
ws.cell(row=ROW_NO_1+4, column=2).alignment = Alignment(horizontal="right", vertical="center")

ws.cell(row=ROW_NO_1+2, column=4, value="거래량")
ws.cell(row=ROW_NO_1+2, column=5).value = v_volume
ws.cell(row=ROW_NO_1+2, column=5).alignment = Alignment(horizontal="right", vertical="center")
ws.cell(row=ROW_NO_1+3, column=4, value="거래대금(억원)")
ws.cell(row=ROW_NO_1+3, column=5).value=v_tr_amt
ws.cell(row=ROW_NO_1+3, column=5).alignment = Alignment(horizontal="right", vertical="center")
ws.cell(row=ROW_NO_1+4, column=4, value="외국인 보유비중(%)")
ws.cell(row=ROW_NO_1+4, column=5).value = v_foreigner
ws.cell(row=ROW_NO_1+4, column=5).alignment = Alignment(horizontal="right", vertical="center")

for row in range(ROW_NO_1+2, ROW_NO_1+5):
    ws.cell(row=row, column=1).fill = PatternFill(fgColor="99ccff", fill_type="solid")
    ws.cell(row=row, column=4).fill = PatternFill(fgColor="99ccff", fill_type="solid")


ws.cell(row=ROW_NO_2, column=1, value="2. 최근 일주일 거래 현황")

ws.cell(row=ROW_NO_2+2, column=1, value='날짜')
ws.cell(row=ROW_NO_2+2, column=2, value='종가')
ws.cell(row=ROW_NO_2+2, column=3, value='등락률')
ws.cell(row=ROW_NO_2+2, column=4, value='거래량')
ws.cell(row=ROW_NO_2+2, column=5, value='거래대금')
for col in range(1, 6):
    ws.cell(row=ROW_NO_2+2, column=col).alignment = Alignment(horizontal="center", vertical="center")
    ws.cell(row=ROW_NO_2+2, column=col).fill = PatternFill(fgColor="99ccff", fill_type="solid")
    
for idx, data in df_recent_price.iterrows():
    ws.cell(row=idx+ROW_NO_2+3, column=1).value = data['날짜']
    ws.cell(row=idx+ROW_NO_2+3, column=1).alignment = Alignment(horizontal="center", vertical="center")
    ws.cell(row=idx+ROW_NO_2+3, column=2).value = data['종가']
    ws.cell(row=idx+ROW_NO_2+3, column=2).alignment = Alignment(horizontal="right", vertical="center")
    ws.cell(row=idx+ROW_NO_2+3, column=3).value = data['등락률']
    ws.cell(row=idx+ROW_NO_2+3, column=3).alignment = Alignment(horizontal="right", vertical="center")
    ws.cell(row=idx+ROW_NO_2+3, column=4).value = data['거래량']
    ws.cell(row=idx+ROW_NO_2+3, column=4).alignment = Alignment(horizontal="right", vertical="center")
    ws.cell(row=idx+ROW_NO_2+3, column=5).value = data['거래대금']
    ws.cell(row=idx+ROW_NO_2+3, column=5).alignment = Alignment(horizontal="right", vertical="center")

ws.cell(row=ROW_NO_3, column=1, value="3. 주가 및 거래량 추이 (최근 6개월)")

""" BEGIN: 데이터 Sheet """

ws_data = wb.create_sheet("DataSheet", 2)        # 2번째 index에 sheet 생성

# 열 너비 지정
ws_data.column_dimensions["A"].width = 15     
ws_data.column_dimensions["B"].width = 15    
ws_data.column_dimensions["C"].width = 15   
ws_data.column_dimensions["D"].width = 15    
ws_data.column_dimensions["E"].width = 25  

# 1줄씩 데이터 넣기
ws_data.append(["날짜", "종가", "등락률", "거래량", "거래대금"])
for index, data in df_period_price.iterrows():
    ws_data.append([data['날짜'], data['종가'], data['등락률'], data['거래량'], data['거래대금']])

""" END: 데이터 Sheet """

value = Reference(ws_data, min_row=1, max_row=max_row+1, min_col=2, max_col=2)
category = Reference(ws_data, min_row=1, max_row=max_row+1, min_col=1)

line_chart = LineChart()
line_chart.set_categories(category)
line_chart.add_data(value, titles_from_data=True)
line_chart.title = "주가(종가) 동향"
line_chart.style = 10
# line_chart.x_axis.title = "날짜"
# line_chart.y_axis.title = "종가"

ws.add_chart(line_chart, "A20")   # 차트 넣을 위치

bar_chart = BarChart()
bar_chart.set_categories(category)
bar_chart.add_data(value, titles_from_data=True)
bar_chart.title = "거래량"
# bar_chart.x_axis.title = "날짜"
# bar_chart.y_axis.title = "거래량"

ws.add_chart(bar_chart, "A33")   # 차트 넣을 위치

# 파일명은 full path로 지정
excel_file = "D:\PythonProject\data-gatherer\stock\data\dse_daily_price.xlsx"
wb.save(excel_file)
wb.close()

### 4. Excel 파일을 PDF로 변환하기

In [136]:
import win32com.client

excel = win32com.client.DispatchEx("Excel.Application")
excel.Visible = False
    
wb = excel.Workbooks.Open(excel_file)
ws = wb.ActiveSheet

# Type=0 "PDF", Type=1 "XPS"
pdf_file = "D:\PythonProject\data-gatherer\sample\data\sample_doc.pdf"
ws.ExportAsFixedFormat(Type=0, Filename=pdf_file)    

wb.application.displayalerts = False  # This stops the popup asking for a save
wb.Close()

excel.Quit()

com_error: (-2147352567, '예외가 발생했습니다.', (0, 'Microsoft Excel', '문서가 저장되지 않았습니다. 문서가 열려 있거나 문서를 저장할 때 오류가 발생했습니다.', 'xlmain11.chm', 0, -2146827284), None)

### 5. 이메일 발송

In [14]:
# Email 수신자 목록

import csv

receiver_file = "D:\PythonProject\data-gatherer\stock\data\email_receiver.csv"
f = open(receiver_file, 'r', encoding='utf-8')
lines = csv.reader(f)
receiver_list = []
for line in lines:
    # print(line)
    receiver_list.append(line[0])
f.close()

email_receivers = ', '.join(receiver_list)  

In [12]:
receiver_list

['stewart@daesung.com', 'kyungho.lim@gmail.com']

In [13]:
email_receivers

'stewart@daesung.com, kyungho.lim@gmail.com'

In [ ]:
""" Send email with attachment """
"""
<< Prerequisites >>
구글 이메일 계정 사용
1. 보안 - 2단계 인증 사용 설정
2. 앱 비밀번호 생성: 앱-메일, 기기-Windows 컴퓨터 선택 => 앱 비밀번호 생성

"""
EMAIL_ADDRESS = "kyungho.lim@gmail.com"
EMAIL_PASSWORD = "zqwsuqdrtmnzvccx"   # Windows 컴퓨터용 앱 비밀번호

import smtplib
# from account import *
from email.message import EmailMessage
import os

msg = EmailMessage()

msg["Subject"] = "대성에너지 일일 주가 동향"+v_ref_date

msg["From"] = EMAIL_ADDRESS

# 수신자가 여러명일 경우 콤마(,)로 구분하여 문자열 생성, msg["To"]에 할당
msg["To"] = email_receivers

# 본문
msg.set_content("대성에너지 일일 주가 동향"+v_ref_date+"입니다. 첨부파일을 다운로드 하십시오.")

"""
MIME type 전체 목록 참조하여 보내는 파일형식에 맞는 maintype과 subtype 지정
=> https://developer.mozilla.org/ko/docs/Web/HTTP/Basics_of_HTTP/MIME_types/Common_types
"""
with open("learning_output/sample_document.pdf", "rb") as f:
    # msg.add_attachment(f.read(), maintype="application", subtype="pdf", filename=f.name)
    file_name = os.path.basename(f.name)
    msg.add_attachment(f.read(), maintype="application", subtype="pdf", filename=file_name)

# 메일 발송
with smtplib.SMTP("smtp.gmail.com", 587) as smtp:   # port no. 587
    smtp.ehlo()         # 연결이 잘 수립되는지 확인
    smtp.starttls()     # 모든 내용이 암호화되어 전송
    smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)   # 로그인
    
    smtp.send_message(msg)